In [ ]:
# =========================================================
# SEÇÃO 1: IMPORTAÇÕES E SETUP GERAL (APENAS PARA ANÁLISE)
# =========================================================

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import friedmanchisquare
import scikit_posthocs as sp
from IPython.display import display, Markdown
from tqdm import tqdm
from scikit_posthocs._critical_values import get_critical_value

# --- Importando as funções compartilhadas necessárias ---
from helper.utils import carregar_serie, dividir_serie_temporal, calcular_metricas

In [ ]:
# =========================================================
# SEÇÃO 2: FUNÇÕES DE GERAÇÃO DE RELATÓRIOS (CAMADA GOLD)
# =========================================================

In [ ]:
def calcular_metricas_finais(df_results):
    """
    Função auxiliar robusta para calcular as métricas a partir do DataFrame de previsões brutas.
    """
    modelos = [col for col in df_results.columns if col not in ['ds', 'y_true', 'dataset', 'horizonte']]
    
    # Dicionário para armazenar dados de treino para o cálculo do MASE
    y_train_dict = {}
    
    # Para cada dataset, carrega os dados de treino e teste originais para o cálculo do MASE
    for dataset_nome in df_results['dataset'].unique():
        horizonte_max_ds = df_results[df_results['dataset'] == dataset_nome]['horizonte'].max()
        serie_original = carregar_serie(dataset_nome)
        percentual_treino = 1 - (horizonte_max_ds / len(serie_original))
        treino, _ = dividir_serie_temporal(serie_original, percentual_treino)
        y_train_dict[dataset_nome] = treino.values

    # Transforma o DataFrame de 'largo' para 'longo' para facilitar a agregação
    df_melted = df_results.melt(id_vars=['ds', 'y_true', 'dataset', 'horizonte'], 
                                value_vars=modelos, var_name='Modelo', value_name='y_pred')
    
    metricas_gerais = []
    # Agrupa para calcular as métricas para cada combinação de dataset, horizonte e modelo
    for (dataset, horizonte, modelo), group in tqdm(df_melted.groupby(['dataset', 'horizonte', 'Modelo']), desc="Calculando Métricas"):
        if not group['y_pred'].isnull().all():
            # Passa a fatia correta dos dados de treino para o MASE
            y_train_correto = y_train_dict[dataset]
            
            # Calcula as métricas
            metricas_dict = calcular_metricas(group['y_true'], group['y_pred'], y_train_correto)
            
            # Adiciona as informações de metadados ao dicionário
            metricas_dict['dataset'] = dataset
            metricas_dict['horizonte'] = horizonte
            metricas_dict['Modelo'] = modelo
            
            metricas_gerais.append(metricas_dict)
    
    df_metricas_final = pd.DataFrame(metricas_gerais)
    # Renomeia colunas para clareza nos relatórios
    df_metricas_final.rename(columns={'RMSE': 'Mean RMSE', 'MAPE(%)': 'Mean MAPE(%)', 'MASE': 'Mean MASE'}, inplace=True)
    return df_metricas_final

In [ ]:
def plotar_evolucao_erro(df_metricas, vetor_horizontes):
    """RELATÓRIO 1: Gera o gráfico de linha da evolução do erro."""
    print("\n--- RELATÓRIO 1: EVOLUÇÃO DO ERRO (RMSE) POR HORIZONTE ---")
    plt.figure(figsize=(14, 7))
    sns.lineplot(data=df_metricas, x='horizonte', y='Mean RMSE', hue='Modelo', style='Modelo', markers=True, dashes=False)
    plt.title("Evolução do Erro (RMSE) com o Aumento do Horizonte", fontsize=16)
    plt.xlabel("Horizonte de Previsão"); plt.ylabel("RMSE Médio"); plt.grid(True)
    if not df_metricas.empty:
        plt.xticks(vetor_horizontes)
    plt.legend(title='Modelo'); plt.show()

In [ ]:
def exibir_desempenho_agregado(df_foco):
    """RELATÓRIO 2: Mostra a tabela de desempenho geral agregado."""
    print("\n--- RELATÓRIO 2: DESEMPENHO GERAL (MÉDIA NO HORIZONTE MAIS LONGO) ---")
    df_agrupado = df_foco.groupby('Modelo')[['Mean RMSE', 'Mean MAPE(%)', 'Mean MASE']].mean()
    display(df_agrupado.style.format('{:.3f}').highlight_min(axis=0, props='background-color: #4285F4; color: white;'))

In [ ]:
def exibir_desempenho_detalhado(df_foco):
    """RELATÓRIO 3: Mostra a tabela de desempenho detalhado por dataset."""
    print("\n--- RELATÓRIO 3: DESEMPENHO DETALHADO POR DATASET (HORIZONTE MAIS LONGO) ---")
    df_reporte_detalhado = df_foco.set_index(['dataset', 'Modelo']).drop(columns=['horizonte'])
    display(df_reporte_detalhado.style.format('{:.3f}'))

In [ ]:
def exibir_tabela_ranking(df_foco):
    """RELATÓRIO 4: Mostra a tabela de ranking e a retorna para uso posterior."""
    print("\n--- RELATÓRIO 4: RANKING DOS MODELOS (BASEADO EM RMSE, HORIZONTE MAIS LONGO) ---")
    df_rank = df_foco.copy()
    df_rank['Rank'] = df_rank.groupby('dataset')['Mean RMSE'].rank().astype(int)
    df_pivot_rank = df_rank.pivot_table(index='dataset', columns='Modelo', values='Rank')
    if len(df_pivot_rank) > 1:
        df_pivot_rank.loc['Média do Rank'] = df_pivot_rank.mean(axis=0)
    display(df_pivot_rank.style.format('{:.1f}').highlight_min(axis=1, props='background-color: #4285F4; color: white;'))
    return df_pivot_rank

In [ ]:
def plotar_diferenca_percentual(df_foco):
    """RELATÓRIO 5: Mostra o gráfico de ganho percentual do modelo híbrido."""
    print("\n--- RELATÓRIO 5: GANHO PERCENTUAL DO MODELO HÍBRIDO (BASEADO EM MAPE, HORIZONTE MAIS LONGO) ---")
    df_pivot_mape = df_foco.pivot_table(index='dataset', columns='Modelo', values='Mean MAPE(%)')
    modelo_referencia_hibrido = 'Híbrido (MIMO)'
    if modelo_referencia_hibrido in df_pivot_mape.columns:
        mape_hibrido = df_pivot_mape[modelo_referencia_hibrido]
        df_pd = pd.DataFrame(index=df_pivot_mape.index)
        for modelo in [m for m in df_pivot_mape.columns if m != modelo_referencia_hibrido]:
            df_pd[f'Ganho sobre {modelo} (%)'] = 100 * (df_pivot_mape[modelo] - mape_hibrido) / df_pivot_mape[modelo]
        ax = df_pd.plot(kind='bar', figsize=(14, 7), grid=True, rot=45); ax.set_ylabel("Melhora Percentual (%)"); ax.set_xlabel("Dataset")
        ax.set_title(f"Diferença Percentual (PD%): Ganho de Performance do {modelo_referencia_hibrido}"); plt.tight_layout(); plt.show()

In [ ]:
def exibir_analise_estatistica_demsar(df_pivot_rank, maior_horizonte):
    """RELATÓRIO 6: Executa e exibe os testes de Friedman e Nemenyi."""
    print("\n\n" + "="*60); print(f"     RELATÓRIO 6: ANÁLISE ESTATÍSTICA GLOBAL (FRIEDMAN + NEMENYI, HORIZONTE {int(maior_horizonte)})"); print("="*60)
    
    # --- CORREÇÃO APLICADA AQUI ---
    # Inicializamos as variáveis como None para garantir que sempre existam.
    p_values_nemenyi = None
    avg_ranks = None
    
    df_rank_data = df_pivot_rank.drop('Média do Rank', errors='ignore')
    if df_rank_data.empty:
        print("AVISO: Tabela de ranking vazia, não foi possível executar a análise estatística.")
        # Retorna os valores nulos
        return p_values_nemenyi, avg_ranks

    try:
        stat, p_value = friedmanchisquare(*[df_rank_data[col].values for col in df_rank_data.columns])
        print(f"\n--- Teste de Friedman ---\np-valor: {p_value:.4f}")

        if p_value < 0.05:
            print("\n**Conclusão: Há uma diferença estatisticamente significativa entre os modelos.**")
            
            print("\n--- Teste Post-hoc de Nemenyi (p-valores par a par) ---")
            df_rank_melted = df_rank_data.reset_index().melt(id_vars='dataset', var_name='Modelo', value_name='Rank')
            p_values_nemenyi = sp.posthoc_nemenyi_friedman(df_rank_melted, melted=True, group_col='Modelo', block_col='dataset', y_col='Rank')
            display(p_values_nemenyi.style.format('{:.3f}').applymap(lambda x: 'background-color: lightgreen' if x < 0.05 else ''))

            # Calcula os ranks médios apenas se o teste for significativo
            avg_ranks = df_pivot_rank.mean(axis=0).drop('Média do Rank', errors='ignore')
        else:
            print("\n**Conclusão: Não há evidência de uma diferença estatística significativa entre os modelos.**")
            
    except Exception as e:
        print(f"AVISO: A análise estatística avançada falhou: {e}")
        
    # A função agora sempre retorna uma tupla, mesmo que os valores sejam None
    return p_values_nemenyi, avg_ranks    

In [ ]:
def plotar_diagrama_diferenca_critica(df_pivot_rank, alpha=0.05):
    """RELATÓRIO ADICIONAL: Gera o Diagrama de Diferença Crítica (CD Plot)."""
    if df_pivot_rank.empty:
        print("AVISO: Tabela de ranking vazia, não é possível gerar o CD Plot.")
        return

    print("\\n--- DIAGRAMA DE DIFERENÇA CRÍTICA (CD PLOT) ---")
    display(Markdown("Este gráfico resume o teste de Nemenyi. Modelos que **NÃO** são significativamente diferentes estão conectados por uma linha horizontal."))
    
    # Extrai os dados de ranking e os nomes dos modelos
    rank_data = df_pivot_rank.drop('Média do Rank', errors='ignore').values
    model_names = df_pivot_rank.columns
    
    # Calcula a diferença crítica (CD)
    # N = número de datasets, k = número de modelos
    N = len(rank_data)
    k = len(model_names)
    
    # Importa a tabela de valores críticos q_alpha do scikit-posthocs
   
    q_alpha = get_critical_value(k, alpha)
    
    cd = q_alpha * np.sqrt(k * (k + 1) / (6 * N))
    
    # Calcula os ranks médios
    avg_ranks = df_pivot_rank.loc['Média do Rank'].sort_values()
    
    # Gera o gráfico usando a função de plotagem do scikit-posthocs
    sp.sign_plot(avg_ranks, cd=cd)
    plt.title(f"Diagrama de Diferença Crítica (Teste de Nemenyi, alpha={alpha})", fontsize=16)
    plt.xlabel("Ranking Médio")
    plt.show()

In [ ]:
# --- NOVA FUNÇÃO DE RELATÓRIO (BASEADA NA TABELA 4 DO ARTIGO) ---
def exibir_tabela_ranking(df_foco, metrica='Mean RMSE'):
    """RELATÓRIO 4: Mostra a tabela de ranking dos modelos."""
    print(f"\n--- RELATÓRIO 4: RANKING DOS MODELOS (BASEADO EM {metrica}) ---")
    df_rank = df_foco.copy()
    rank_col_name = f'Rank_{metrica}'
    df_rank[rank_col_name] = df_rank.groupby('dataset')[metrica].rank().astype(int)
    df_pivot_rank = df_rank.pivot_table(index='dataset', columns='Modelo', values=rank_col_name)
    if len(df_pivot_rank) > 1:
        df_pivot_rank.loc['Média do Rank'] = df_pivot_rank.mean(axis=0)
    display(df_pivot_rank.style.format('{:.1f}').highlight_min(axis=1, props='background-color: #4285F4; color: white;'))
    return df_pivot_rank

In [ ]:
def exibir_ranking_mape_artigo(df_foco):
    """RELATÓRIO EXTRA 1: Gera a tabela de ranking por MAPE, no estilo da Tabela 4 do artigo."""
    print("\n\n" + "="*60)
    print("     RELATÓRIO ADICIONAL: TABELA DE RANKING POR MAPE (ESTILO ARTIGO)")
    print("="*60)
    
    df_rank_mape = df_foco.copy()
    df_rank_mape['Rank_MAPE'] = df_rank_mape.groupby('dataset')['Mean MAPE(%)'].rank().astype(int)
    df_pivot = df_rank_mape.pivot_table(index='Modelo', columns='dataset', values='Rank_MAPE')
    
    # Calcula a Média e Mediana do Rank para cada modelo
    df_pivot['Média'] = df_pivot.mean(axis=1)
    df_pivot['Mediana'] = df_pivot.median(axis=1)
    
    def highlight_top3(s):
        is_top3 = s <= 3
        return ['background-color: blue' if v else '' for v in is_top3]

    styled_pivot = (df_pivot.style
                    .format('{:.1f}')
                    .apply(highlight_top3, subset=pd.IndexSlice[:, [c for c in df_pivot.columns if c not in ['Média', 'Mediana']]])
                    .set_caption("Ranking dos modelos por dataset baseado no MAPE (1 = Melhor)."))
    
    display(styled_pivot)

In [ ]:
# --- NOVA FUNÇÃO DE RELATÓRIO (BASEADA NA FIGURA 9 DO ARTIGO) ---
def plotar_pd_agregado(df_foco):
    """RELATÓRIO EXTRA 2: Gera o gráfico de Diferença Percentual (PD%) agregado."""
    print("\n\n" + "="*60)
    print("     RELATÓRIO ADICIONAL: GRÁFICO DE DIFERENÇA PERCENTUAL AGREGADO (ESTILO FIGURA 9)")
    print("="*60)

    modelo_referencia_hibrido = 'Híbrido (MIMO)'
    # Calcula o MAPE médio para cada modelo, em todos os datasets
    df_mean_mape = df_foco.groupby('Modelo')['Mean MAPE(%)'].mean()

    if modelo_referencia_hibrido in df_mean_mape.index:
        mape_hibrido = df_mean_mape[modelo_referencia_hibrido]
        pd_values = []
        for modelo, mape_medio in df_mean_mape.items():
            if modelo != modelo_referencia_hibrido:
                pd_value = 100 * (mape_medio - mape_hibrido) / mape_medio
                pd_values.append({'Modelo': modelo, 'PD(%)': pd_value})
        
        if pd_values:
            df_pd = pd.DataFrame(pd_values).sort_values(by='PD(%)')
            
            plt.figure(figsize=(14, 8))
            ax = sns.barplot(x='Modelo', y='PD(%)', data=df_pd, palette='viridis', hue='Modelo', legend=False)
            ax.set_title(f"Diferença Percentual (PD%) Agregada do {modelo_referencia_hibrido}", fontsize=16)
            ax.set_ylabel("Melhora Percentual (%)")
            ax.set_xlabel("Modelo Competidor")
            plt.grid(axis='x', linestyle='--', alpha=0.7)
            plt.tight_layout()
            plt.show()

In [ ]:
# =========================================================
# SEÇÃO 3: FUNÇÕES DE GERAÇÃO DE RELATÓRIOS (CAMADA GOLD)
# =========================================================

In [ ]:
def gerar_suite_completa_de_relatorios(output_file, vetor_horizontes):
    """Função principal que orquestra a geração de todos os relatórios a partir da camada Silver."""
    print("="*60); print("     INICIANDO GERAÇÃO DA SUÍTE COMPLETA DE RELATÓRIOS (CAMADA GOLD)"); print("="*60)
    try:
        # 1. Carrega os dados da camada Silver
        df_results = pd.read_csv(output_file)
        
        # 2. Processa as métricas
        df_metricas_final = calcular_metricas_finais(df_results)
        
        # 3. Gera os relatórios
        plotar_evolucao_erro(df_metricas_final, vetor_horizontes)
        
        maior_horizonte = df_metricas_final['horizonte'].max()
        df_foco_maior_h = df_metricas_final[df_metricas_final['horizonte'] == maior_horizonte]
        
        display(Markdown(f"### Análises Detalhadas para o Horizonte Mais Longo ({int(maior_horizonte)} passos)"))
        
        exibir_desempenho_agregado(df_foco_maior_h)
        exibir_desempenho_detalhado(df_foco_maior_h)
        df_pivot_rank = exibir_tabela_ranking(df_foco_maior_h)
        
        # Análise estatística
        p_values_nemenyi, avg_ranks = exibir_analise_estatistica_demsar(df_pivot_rank, maior_horizonte)
        plotar_diagrama_diferenca_critica(p_values_nemenyi, avg_ranks) # Adapte para usar os outputs corretos
        
        print("\n" + "="*60); print("     SUÍTE DE RELATÓRIOS GERADA COM SUCESSO!"); print("="*60)

    except FileNotFoundError:
        print(f"\\nERRO: Arquivo da camada Silver '{output_file}' não encontrado. Execute primeiro o script '01_run_experiments'.")
    except Exception as e:
        import traceback
        print(f"Ocorreu um erro ao gerar os relatórios: {e}")
        traceback.print_exc()

In [ ]:
# =========================================================
# SEÇÃO 4: EXECUÇÃO DA CAMADA GOLD
# =========================================================

In [ ]:
silver_layer_file = "./data/silver/resultados_completos.csv"
VETOR_DE_HORIZONTES = [10, 12, 15, 24]

gerar_suite_completa_de_relatorios(silver_layer_file, VETOR_DE_HORIZONTES)